In [1]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [2]:
import logging
import pandas as pd

from general_functions.return_account_ids import return_account_ids
from general_functions.constants import return_api_url
from general_functions.call_api_with_account_id import send_to_innkeepr_api_paginated

In [8]:
customer = "Nikin" #6617da9c01c2ab3bd12c63cb
date = "2025-09-01T00:00:00.000Z"
url = return_api_url()
print(f"url = {url}")
account_id = return_account_ids()
account_id = [acc["id"] for acc in account_id if acc["name"] == customer]
account_id = account_id[0]

In [13]:
try:
    sessions = pd.read_parquet(f"DataChecks/sessions/test_sessions_{customer}_{date}.parquet")
except FileNotFoundError:
    sessions_response = send_to_innkeepr_api_paginated( #TODO: try out with limit
        f"{url}/sessions/query",
        account_id,
        {"created":{"$gte":date}},
        logging
    )
    sessions = pd.json_normalize(sessions_response)
    sessions.to_parquet(f"DataChecks/sessions/test_sessions_{customer}_{date}.parquet")
sessions

# Some Checks

## check signals

In [ ]:
signals = send_to_innkeepr_api_paginated(
    f"{url}/treatments/query",
    account_id,
    {"externalId":sessions["campaign.utm_id"].dropna().unique().tolist()},
    logging
)

signals = pd.json_normalize(signals)
signals

In [ ]:
signals = send_to_innkeepr_api_paginated(
    f"{url}/signals/query",
    account_id,
    {"externalId":sessions["campaign.gclid"].dropna().unique().tolist()},
    logging
)

signals = pd.json_normalize(signals)
signals

## check with conversions

In [14]:
from general_functions.datetime_helper import transform_date_to_timestamp_milliseconds

test = sessions[sessions["campaign.ad_id"].isnull()==False]
meta_sessions = test["sessionId"].dropna().unique().tolist()
min_date = sessions["created"].min()
max_date = sessions["created"].max()
min_date = pd.to_datetime(min_date).strftime("%Y%m%d")
max_date = pd.to_datetime(max_date).strftime("%Y%m%d")
content = {
     "created": {
                     "$gte": transform_date_to_timestamp_milliseconds(min_date),
                     "$lte": transform_date_to_timestamp_milliseconds(max_date),
               }
    }
conversions = send_to_innkeepr_api_paginated(
    f"{url}/conversions/query",
    account_id,
    content,
    logging
    
)

conversions = pd.json_normalize(conversions)
conversions

In [15]:
conversions_matches = conversions[conversions["sessionId"].isin(meta_sessions)]
conversions_matches

In [ ]:
externalIds = sessions[sessions["sessionId"].isin(conversions_matches["sessionId"])]["campaign.utm_id"].dropna().unique().tolist()
len(externalIds), externalIds

In [ ]:
signals = send_to_innkeepr_api_paginated(
    f"{url}/signals/query",
    account_id,
    {"externalId":externalIds},
    logging
)

signals = pd.json_normalize(signals)
signals

In [ ]:
treatments = send_to_innkeepr_api_paginated(
    f"{url}/treatments/query",
    account_id,
    {"id":signals["relates_to.treatment"].dropna().unique().tolist()},
    logging
)
treatments = pd.json_normalize(treatments)
treatments

# chech sessions and gclids

In [18]:
sessions.columns, conversions.columns

In [19]:
sessions_with_conversions = sessions[["id","created","sessionId","anonymousId","campaign.gclid"]]
sessions_with_conversions = pd.merge(sessions_with_conversions, conversions_matches[["sessionId","created","name"]], how="left", on="sessionId", suffixes=("", "_conv"))
sessions_with_conversions

In [20]:
sessions_with_conversions = sessions_with_conversions[sessions_with_conversions["name"]=="checkout_completed"]
sessions_with_conversions

In [21]:
sessions_with_conversions["date"] = pd.to_datetime(sessions_with_conversions["created"]).dt.date

sessions_with_conversions.groupby("date")["campaign.gclid"].nunique()

In [23]:
17*30